# Mantenimiento Predictivo

# Acceso a una PI AF Database
Requiere tener instalado: pythonnet
http://pythonnet.github.io/

Confirmar dónde se encuentran las dlls: OSIsoft.AFSDK.dll en su máquina.
# 0- Preparación del Ambiente

In [ ]:
#!pip install pythonnet

In [1]:
# sys module provides access to some variables used or maintained by the interpreter and to functions that interact 
# strongly with the interpreter. It is always available. https://docs.python.org/3/library/sys.html
# clr es la forma de declarar el package pythonnet (.net para python)
import sys
import clr

In [2]:
sys.path.append(r'C:\Program Files (x86)\PIPC\AF\PublicAssemblies\4.0')  
clr.AddReference('OSIsoft.AFSDK')

In [3]:
from OSIsoft.AF import *
from OSIsoft.AF.PI import *
from OSIsoft.AF.Asset import *
from OSIsoft.AF.Data import *
from OSIsoft.AF.Time import *
from OSIsoft.AF.UnitsOfMeasure import *
from OSIsoft.AF.Search import *

In [4]:
#initializes a C# list: from System.Collections.Generic import List
# Implicit loading is deprecated. Please use clr.AddReference('System.Collections').
clr.AddReference('System.Collections')
from System.Collections.Generic import List

In [5]:
from datetime import datetime, timedelta

In [6]:
import pandas as pd

# 1- Acceso a una AF Database

In [7]:
#https://docs.osisoft.com/bundle/af-sdk/page/html/pisystem-hierarchy.htm
afservers = PISystems()
print('Tipo de Datos:', type(afservers))
iterator = afservers.GetEnumerator()
for server in iterator:
    print(server.Name)

Tipo de Datos: <class 'OSIsoft.AF.PISystems'>
192.168.103.66
PARCCVWPIEDA01
QARCCVWPIE01


In [8]:
# Objeto para la base de datos que busco consultar
# https://docs.osisoft.com/bundle/af-sdk/page/html/T_OSIsoft_AF_PISystem.htm
#afserver = afservers.DefaultPISystem
AFServer = afservers['PARCCVWPIEDA01']
#AFServer = afservers['QARCCVWPIE01']
print(AFServer.Name, AFServer.UniqueID, 'Clase:', type(AFServer))

PARCCVWPIEDA01 69941c34-7d8c-491f-b5b8-70bb5f66436f Clase: <class 'OSIsoft.AF.PISystem'>


In [9]:
# Acceso a una base de datos que no sea la default:
# afserver.Databases: https://docs.osisoft.com/bundle/af-sdk/page/html/T_OSIsoft_AF_AFDatabases.htm
# Databases.Database:
DB = AFServer.Databases.DefaultDatabase
print(DB.Name, type(AFServer))
DB = AFServer.Databases.get_Item("DOF")
print ('Database Name: {0}'.format(DB.Name), type(DB))

DOF <class 'OSIsoft.AF.PISystem'>
Database Name: DOF <class 'OSIsoft.AF.AFDatabase'>


# 2- Recorrer la estructura de Assets y Atributos

In [15]:
type(DB.Elements)

OSIsoft.AF.Asset.AFElements

In [10]:
for element in DB.Elements:
    print('Nivel 1:', element.Name)
    # Puedo guardar los atributos en una lista:
    asset_lst = list(element.Elements)
    for subelement in element.Elements:
        if subelement.Name == 'El Mangrullo':
            print('Nivel 2:', subelement.Name, type(subelement.Name))
            # Puedo guardar los atributos en una lista:
            attr_lst = list(subelement.Attributes)
            # No encontré otra manera de acceder a un atributo, más que esta:
            attr = subelement.Attributes.GetItem('Promedio Entrega Gas 9300 (PlanGasAr)', True, True, True)
            print('Attribute:', attr, attr.GetValue(), attr.get_DefaultUOM())
            i = 0
            print('Primeros 10 Atributos y su valor actual:')
            for attribute in subelement.Attributes:
                if i < 10:
                    print(attribute.Name, attribute.GetValue().Timestamp, attribute.GetValue(), attribute.DisplayUOM, attribute.Data)
                    i +=1

Nivel 1: PAMPA ENERGIA S.A.
Nivel 2: El Mangrullo <class 'str'>
Attribute: Promedio Entrega Gas 9300 (PlanGasAr) 6131,75041992187 Miles Metros Cubicos
Primeros 10 Atributos y su valor actual:
Plan Gas. Promedio Acumulado Venta Gas@9300 Participación 20/2/2022 00:00:00 6000 Miles Metros Cubicos OSIsoft.AF.Data.AFData
Promedio Entrega Gas 9300 (PlanGasAr) 20/2/2022 00:00:00 6131,75041992187 Miles Metros Cubicos OSIsoft.AF.Data.AFData
Venta Prom. Gas 9300 vs Plan Anual en Participación (b) 20/2/2022 00:00:00 246,857568359375 Miles Metros Cubicos OSIsoft.AF.Data.AFData
Venta Gas 9300 vs Plan Gas Mes en Participación (b) 20/2/2022 00:00:00 245,76220703125 Miles Metros Cubicos OSIsoft.AF.Data.AFData
FC 20/2/2022 00:00:00 1,00571181547394 None OSIsoft.AF.Data.AFData
Producción Balance Mensual de Petróleo 1/1/2022 00:00:00 6,6248064516129 Cubic Meter per Day OSIsoft.AF.Data.AFData
Promedio Balance Diario Gas @9300 Pozos Tight 20/2/2022 00:00:00 74,36050855 Miles Metros Cubicos OSIsoft.AF.Data.

# 3- Consultas de Datos

## Armo los objetos con assets y sus atributos

In [12]:
# 1° Objeto del arbol:
root = DB.Elements.get_Item(0)
root.Name

'PAMPA ENERGIA S.A.'

In [38]:
# Busco los objetos que necesito:
asset_lst = list(root.Elements)
EMA = asset_lst[3]
print(EMA.Name)
EMA_Inst = list(asset_lst[3].Elements)[0]
print(EMA_Inst.Name)
EMA_PTG = list(EMA_Inst.Elements)[3]
print(EMA_PTG.Name)
EMA_PTG_Comp = list(EMA_PTG.Elements)[0]
print(EMA_PTG_Comp.Name)
EMA_PTG_K_01 = list(EMA_PTG_Comp.Elements)[0]
print(EMA_PTG_K_01.Name)
EMA_PTG_K_02 = list(EMA_PTG_Comp.Elements)[1]
EMA_PTG_K_03 = list(EMA_PTG_Comp.Elements)[2]
EMA_PTG_K_04 = list(EMA_PTG_Comp.Elements)[3]
EMA_PTG_K_05 = list(EMA_PTG_Comp.Elements)[4]
EMA_PTG_K_06 = list(EMA_PTG_Comp.Elements)[5]
EMA_PTG_K_07 = list(EMA_PTG_Comp.Elements)[6]
EMA_PTG_K_08 = list(EMA_PTG_Comp.Elements)[7]
EMA_PTG_K_09 = list(EMA_PTG_Comp.Elements)[8]

El Mangrullo
Instalaciones
PTG EMAN
Compresores
K-01


In [166]:
# Puedo armar otra lista con los atributos de un asset:
attr_lst = list(asset_lst[13].Attributes)
print('Nivel 1, ejemplo de atributo:', attr_lst[0].Name, attr_lst[0].GetValue().Timestamp, attr_lst[0].GetValue(), 
      attr_lst[0].DisplayUOM, '- Objeto padre:', attr_lst[0].Element.Name)

Nivel 1, ejemplo de atributo: Plan Gas. Promedio Acumulado Venta Gas@9300 Participación 31/1/2022 00:00:00 600,00001152 Miles Metros Cubicos - Objeto padre: Sierra Chata


## Consultas - Definición de Rango de Tiempo de la consulta

In [44]:
# Busco el atributo:
attr = EMA_PTG_K_01.Attributes.GetItem('Carga Motriz Compresor', True, True, True)
print('Attribute:', attr, attr.GetValue().Timestamp, attr.GetValue(), attr.get_DefaultUOM())

Attribute: Carga Motriz Compresor 21/2/2022 16:26:31 91,748046875 ampere


In [49]:
attr_lst = ['Carga Motriz Compresor', 'Presion Descarga Cil. 1y3 Compresor', 'Presion Descarga Cil. 2y4 Compresor',
           'Presion Succion Compresor', 'RPMs Instantanea']

In [51]:
################ INTERPOLATED VALUES ###########################
# Definir el intervalo de consulta:

# A AFTimeSpan represents a time interval (duration of time or elapsed time) that is measured as a positive or negative 
# number of years, months, days, hours, minutes, seconds, milliseconds, and fractions of a millisecond. 
# Create AFTimeSpan using constructor

now = AFTime.Now
span = AFTimeSpan.Parse('5m')
print("TimeSpan ", span)

TimeSpan  5m


In [70]:
# Defino un rango de tiempo para consultas de datos históricos
#### Tiempo Relativo:
timerange_start = AFTime('*')
#timerange_span = AFTimeSpan.Parse('-174d') # 1h, 1d, para restar tiempo: -1m, -1h, etc
#timerange_end = AFTime.op_Addition(timerange_start, timerange_span)
timerange_end = AFTime('2021-09-01 00:00:00')
timerange = AFTimeRange(timerange_start, timerange_end)
print(timerange_start, timerange_end, timerange_span)

21/2/2022 16:56:12 1/9/2021 00:00:00 -174d


In [71]:
#df_hist = pd.DataFrame(columns=['Asset', 'Attribute', 'Timestamp', 'Value', 'UOM'])
df_row = list()
pdlist = [] # en esta lista quedan todos los df para luego concatenarlos

for element in list(EMA_PTG_Comp.Elements):
    for attr in attr_lst:
        attr_obj = element.Attributes.GetItem(attr, True, True, True)
        values_lst = attr_obj.Data.InterpolatedValues(timerange, span, attr_obj.DisplayUOM, "", False)
        for val in values_lst:
            if True:
                #if val.IsGood:
                df_row.append([val.Attribute.Element.Name, val.Attribute.Name, val.Timestamp, val.Value, 
                                  val.Attribute.DisplayUOM])
df_hist = pd.DataFrame(df_row, columns=['Asset', 'Attribute', 'Timestamp', 'Value', 'UOM'])
df_hist

,Asset,Attribute,Timestamp,Value,UOM
0,K-01,Carga Motriz Compresor,21/2/2022 16:55:00,92.138672,ampere
1,K-01,Carga Motriz Compresor,21/2/2022 16:50:00,91.845703,ampere
2,K-01,Carga Motriz Compresor,21/2/2022 16:45:00,91.748047,ampere
3,K-01,Carga Motriz Compresor,21/2/2022 16:40:00,91.845703,ampere
4,K-01,Carga Motriz Compresor,21/2/2022 16:35:00,91.845703,ampere
...,...,...,...,...,...
2251255,K-09,RPMs Instantanea,1/9/2021 00:20:00,1159.565674,revolution per minute
2251256,K-09,RPMs Instantanea,1/9/2021 00:15:00,1160.199097,revolution per minute
2251257,K-09,RPMs Instantanea,1/9/2021 00:10:00,1158.687866,revolution per minute
2251258,K-09,RPMs Instantanea,1/9/2021 00:05:00,1160.011353,revolution per minute


In [67]:
path_data = 'C:\\Users\\COLMO\\OneDrive\\0-Data Science\\data_science\\data_pampa'
#path_data = 'C:\\Users\\Usuario\\OneDrive\\0-Data Science\\data_science\\data_pampa'

In [ ]:
output_file = 'EMA_compresores_5min_AF_ok.csv'
df_hist.to_csv(path_data + '\\' + output_file, sep=';', encoding='UTF-8', decimal = '.')

In [48]:
# Realizar la consulta:

# When a positive interval (span) is specified, the interval calculation begins at the earliest bounding time in the 
# timeRange and applies the interval repeatedly in time ascending direction to generate the calculation intervals.
# If a negative interval is specified, the interval calculation begins at the latest bounding time in the timeRange and 
# applies the interval repeatedly in time descending direction to generate the calculation intervals. Note that the order of values returned will still be reflected by the timeRange, regardless of the interval sign.

values_lst = attr.Data.InterpolatedValues(timerange, span, attr.DisplayUOM, "", False)
# Valores:
print('Elemento:', attr.Element.Name, 'Attribute:', attr.Name, 'Span:', span, 'Time Range:', timerange)
for val in values_lst:
    print(val.Timestamp, val.IsGood, val.Value)

Elemento: K-01 Attribute: Carga Motriz Compresor Span: 5m Time Range: 21/2/2022 16:28:06 - 21/2/2022 15:28:06
21/2/2022 16:28:06 True 91.748046875
21/2/2022 16:23:06 True 91.943359375
21/2/2022 16:18:06 True 91.943359375
21/2/2022 16:13:06 True 91.650390625
21/2/2022 16:08:06 True 91.552734375
21/2/2022 16:03:06 True 91.943359375
21/2/2022 15:58:06 True 91.650390625
21/2/2022 15:53:06 True 91.748046875
21/2/2022 15:48:06 True 91.748046875
21/2/2022 15:43:06 True 91.748046875
21/2/2022 15:38:06 True 91.50390625
21/2/2022 15:33:06 True 91.69921875
21/2/2022 15:28:06 True 91.455078125


In [175]:
add_1_hour = timedelta(hours=1)

dataset_1 = list()

for val in values_lst:
    if val.IsGood:
        timestamp = datetime(val.Timestamp.LocalTime.Year, val.Timestamp.LocalTime.Month, val.Timestamp.LocalTime.Day, 
                            val.Timestamp.LocalTime.Hour, val.Timestamp.LocalTime.Minute, val.Timestamp.LocalTime.Second)
        dataset_1.append([val.Attribute.Element.Name, val.Attribute.Name, val.Timestamp, val.Value, val.Attribute.DisplayUOM])

In [176]:
df = pd.DataFrame(dataset_1, columns=['Asset', 'Attribute', 'Timestamp', 'Value', 'UOM'])
df

,Asset,Attribute,Timestamp,Value,UOM
0,Sierra Chata,Caudal Instantáneo Area,1/2/2022 15:08:12,2090.753931,Miles Metros Cubicos
1,Sierra Chata,Caudal Instantáneo Area,1/2/2022 15:08:04,2090.458989,Miles Metros Cubicos
2,Sierra Chata,Caudal Instantáneo Area,1/2/2022 15:07:59,2088.224236,Miles Metros Cubicos
3,Sierra Chata,Caudal Instantáneo Area,1/2/2022 15:07:57,2088.178801,Miles Metros Cubicos
4,Sierra Chata,Caudal Instantáneo Area,1/2/2022 15:07:54,2089.928728,Miles Metros Cubicos
5,Sierra Chata,Caudal Instantáneo Area,1/2/2022 15:07:49,2098.917636,Miles Metros Cubicos
6,Sierra Chata,Caudal Instantáneo Area,1/2/2022 15:07:44,2104.139022,Miles Metros Cubicos
7,Sierra Chata,Caudal Instantáneo Area,1/2/2022 15:07:42,2104.306879,Miles Metros Cubicos
8,Sierra Chata,Caudal Instantáneo Area,1/2/2022 15:07:34,2098.890522,Miles Metros Cubicos
9,Sierra Chata,Caudal Instantáneo Area,1/2/2022 15:07:29,2098.855289,Miles Metros Cubicos


In [39]:
# Defino un rango de tiempo para consultas de datos históricos

#myTimeRange.StartTime = New AFTime(DateTime.UtcNow.AddHours(-1))
#myTimeRange.EndTime = AFTime.Now
#### Tiempo Absoluto:
#timerange_start = AFTime('2022-01-28 00:00:00')
#### Tiempo Relativo:
timerange_start = AFTime('*')
timerange_span = AFTimeSpan.Parse('-5m') # 1h, 1d, para restar tiempo: -1m, -1h, etc
timerange_end = AFTime.op_Addition(timerange_start, timerange_span)
timerange = AFTimeRange(timerange_start, timerange_end)
print(timerange_start, timerange_end, timerange_span)

#timerange = AFTimeRange("*-31d", "*-30d")
#print(type(timerange), timerange.StartTime, timerange.EndTime, timerange.Span) # Span parece que es en días.

21/2/2022 16:24:08 21/2/2022 16:19:08 -5m


## Consulta de Valores Guardados (no interpolados)

In [174]:
################ RECORDED VALUES ##########################
# https://docs.osisoft.com/bundle/af-sdk/page/html/T_OSIsoft_AF_Data_AFData.htm: 
# The AFData object is associated with a single AFAttribute and is used to retrieve and set extended historical data. 
# It is accessed through the Data property of an AFAttribute.
values_lst = attr.Data.RecordedValues(timerange, AFBoundaryType.Inside, attr.DisplayUOM, "", False, 24)
# Valores:
print('Elemento:', attr.Element.Name, 'Attribute:', attr.Name, 'Time Range:', timerange)
for val in values_lst:
    print(val.Timestamp, val.IsGood, val.Value)

Elemento: Sierra Chata Attribute: Caudal Instantáneo Area Time Range: 1/2/2022 15:08:18 - 1/2/2022 15:07:18
1/2/2022 15:08:12 True 2090.7539309638023
1/2/2022 15:08:04 True 2090.458989040634
1/2/2022 15:07:59 True 2088.2242362918523
1/2/2022 15:07:57 True 2088.1788011490903
1/2/2022 15:07:54 True 2089.9287282441674
1/2/2022 15:07:49 True 2098.917635505719
1/2/2022 15:07:44 True 2104.139021712268
1/2/2022 15:07:42 True 2104.306878703034
1/2/2022 15:07:34 True 2098.8905217051506
1/2/2022 15:07:29 True 2098.8552889497964
1/2/2022 15:07:27 True 2097.4498923020205
1/2/2022 15:07:24 True 2098.7008290944195
1/2/2022 15:07:19 True 2095.989003284382


## Consulta de Valores Interpolados

In [41]:
################ INTERPOLATED VALUES ###########################
# Definir el intervalo de consulta:

# A AFTimeSpan represents a time interval (duration of time or elapsed time) that is measured as a positive or negative 
# number of years, months, days, hours, minutes, seconds, milliseconds, and fractions of a millisecond. 
# Create AFTimeSpan using constructor

now = AFTime.Now
span = AFTimeSpan.Parse('5m')
print("TimeSpan ", span)

TimeSpan  5m


In [47]:
# Defino un rango de tiempo para consultas de datos históricos

#myTimeRange.StartTime = New AFTime(DateTime.UtcNow.AddHours(-1))
#myTimeRange.EndTime = AFTime.Now
#### Tiempo Absoluto:
#timerange_start = AFTime('2022-01-28 00:00:00')
#### Tiempo Relativo:
timerange_start = AFTime('*')
timerange_span = AFTimeSpan.Parse('-1h') # 1h, 1d, para restar tiempo: -1m, -1h, etc
timerange_end = AFTime.op_Addition(timerange_start, timerange_span)
timerange = AFTimeRange(timerange_start, timerange_end)
print(timerange_start, timerange_end, timerange_span)

#timerange = AFTimeRange("*-31d", "*-30d")
#print(type(timerange), timerange.StartTime, timerange.EndTime, timerange.Span) # Span parece que es en días.

21/2/2022 16:28:06 21/2/2022 15:28:06 -1hr


In [48]:
# Realizar la consulta:

# When a positive interval (span) is specified, the interval calculation begins at the earliest bounding time in the 
# timeRange and applies the interval repeatedly in time ascending direction to generate the calculation intervals.
# If a negative interval is specified, the interval calculation begins at the latest bounding time in the timeRange and 
# applies the interval repeatedly in time descending direction to generate the calculation intervals. Note that the order of values returned will still be reflected by the timeRange, regardless of the interval sign.

values_lst = attr.Data.InterpolatedValues(timerange, span, attr.DisplayUOM, "", False)
# Valores:
print('Elemento:', attr.Element.Name, 'Attribute:', attr.Name, 'Span:', span, 'Time Range:', timerange)
for val in values_lst:
    print(val.Timestamp, val.IsGood, val.Value)

Elemento: K-01 Attribute: Carga Motriz Compresor Span: 5m Time Range: 21/2/2022 16:28:06 - 21/2/2022 15:28:06
21/2/2022 16:28:06 True 91.748046875
21/2/2022 16:23:06 True 91.943359375
21/2/2022 16:18:06 True 91.943359375
21/2/2022 16:13:06 True 91.650390625
21/2/2022 16:08:06 True 91.552734375
21/2/2022 16:03:06 True 91.943359375
21/2/2022 15:58:06 True 91.650390625
21/2/2022 15:53:06 True 91.748046875
21/2/2022 15:48:06 True 91.748046875
21/2/2022 15:43:06 True 91.748046875
21/2/2022 15:38:06 True 91.50390625
21/2/2022 15:33:06 True 91.69921875
21/2/2022 15:28:06 True 91.455078125


## Consulta de Valores Agregados

In [185]:
timerange = AFTimeRange("*-3d", "*")
span = AFTimeSpan.Parse('1d')
# Data.Summaries devuelve un iterable, por eso me quedo en el primer elemento (que es de tipo AFValues), sino, usar Summary()
# https://docs.osisoft.com/bundle/af-sdk/page/html/T_OSIsoft_AF_Data_AFCalculationBasis.htm
summaries = list(attr.Data.Summaries(timerange, span, AFSummaryTypes.Average, AFCalculationBasis.TimeWeighted, 
                                        AFTimestampCalculation.Auto))[0].Value
#summaries = attr.Data.Summary(timerange, AFSummaryTypes.Average, AFCalculationBasis.TimeWeighted, AFTimestampCalculation.Auto)
print('Elemento:', attr.Element.Name, 'Atributo:', attr.Name, 'Intervalo:', timerange, 'Span', span, 'Average')
for summ in summaries:
    # Cuando uso Summary() no tiene el atributo Timestamp print(summ.Key, summ.Value)
    print(summ.Timestamp, summ.Value)

Elemento: Sierra Chata Atributo: Caudal Instantáneo Area Intervalo: 29/1/2022 18:56:03 - 1/2/2022 18:56:03 Span 1d Average
29/1/2022 18:56:03 2002.6858600976068
30/1/2022 18:56:03 2034.53307924541
31/1/2022 18:56:03 2083.6623124662365


## Convertir los datos a un Pandas Database

In [175]:
add_1_hour = timedelta(hours=1)

dataset_1 = list()

for val in values_lst:
    if val.IsGood:
        timestamp = datetime(val.Timestamp.LocalTime.Year, val.Timestamp.LocalTime.Month, val.Timestamp.LocalTime.Day, 
                            val.Timestamp.LocalTime.Hour, val.Timestamp.LocalTime.Minute, val.Timestamp.LocalTime.Second)
        dataset_1.append([val.Attribute.Element.Name, val.Attribute.Name, val.Timestamp, val.Value, val.Attribute.DisplayUOM])

In [176]:
df = pd.DataFrame(dataset_1, columns=['Asset', 'Attribute', 'Timestamp', 'Value', 'UOM'])
df

,Asset,Attribute,Timestamp,Value,UOM
0,Sierra Chata,Caudal Instantáneo Area,1/2/2022 15:08:12,2090.753931,Miles Metros Cubicos
1,Sierra Chata,Caudal Instantáneo Area,1/2/2022 15:08:04,2090.458989,Miles Metros Cubicos
2,Sierra Chata,Caudal Instantáneo Area,1/2/2022 15:07:59,2088.224236,Miles Metros Cubicos
3,Sierra Chata,Caudal Instantáneo Area,1/2/2022 15:07:57,2088.178801,Miles Metros Cubicos
4,Sierra Chata,Caudal Instantáneo Area,1/2/2022 15:07:54,2089.928728,Miles Metros Cubicos
5,Sierra Chata,Caudal Instantáneo Area,1/2/2022 15:07:49,2098.917636,Miles Metros Cubicos
6,Sierra Chata,Caudal Instantáneo Area,1/2/2022 15:07:44,2104.139022,Miles Metros Cubicos
7,Sierra Chata,Caudal Instantáneo Area,1/2/2022 15:07:42,2104.306879,Miles Metros Cubicos
8,Sierra Chata,Caudal Instantáneo Area,1/2/2022 15:07:34,2098.890522,Miles Metros Cubicos
9,Sierra Chata,Caudal Instantáneo Area,1/2/2022 15:07:29,2098.855289,Miles Metros Cubicos


# Otra Forma no tan clara

In [108]:
# GetValues(AFTimeRange timeRange, int numberOfValues, UOM desiredUOM) 
# numberOfValues: If 0, then all recorded values within the timeRange will be returned with an interpolated value at the start
# and end time, if possible. If less than zero: will return evenly spaced interpolated values across the timeRange, 
# with a value returned at both end points of the time range. For example, specifying -25 over a 24 hour period will produce 
# an hourly value.
# If the number of values requested is greater than zero, the method will behave like the PlotValues method.
values = attr.GetValues(timerange, 0, attr.DisplayUOM)
values_lst = list(values)

In [109]:
# OJO: los datos aparecen del más reciente al más antiguo:
for value in values_lst:
    print(value.Attribute.Element, value.Attribute, value.Attribute.DisplayUOM)
    att_UOM = value.Attribute.DisplayUOM
    print(value.Timestamp, value.Value, att_UOM)

Sierra Chata Caudal Instantáneo Area Miles Metros Cubicos
1/2/2022 14:29:49 2074.6775240898132 Miles Metros Cubicos
Sierra Chata Caudal Instantáneo Area Miles Metros Cubicos
1/2/2022 14:29:44 2074.6775240898132 Miles Metros Cubicos
Sierra Chata Caudal Instantáneo Area Miles Metros Cubicos
1/2/2022 14:29:42 2074.6791453988267 Miles Metros Cubicos
Sierra Chata Caudal Instantáneo Area Miles Metros Cubicos
1/2/2022 14:29:34 2077.2211876511574 Miles Metros Cubicos
Sierra Chata Caudal Instantáneo Area Miles Metros Cubicos
1/2/2022 14:29:29 2077.8977283798977 Miles Metros Cubicos
Sierra Chata Caudal Instantáneo Area Miles Metros Cubicos
1/2/2022 14:29:27 2047.7712358088943 Miles Metros Cubicos
Sierra Chata Caudal Instantáneo Area Miles Metros Cubicos
1/2/2022 14:29:24 2049.5392437041837 Miles Metros Cubicos
Sierra Chata Caudal Instantáneo Area Miles Metros Cubicos
1/2/2022 14:29:19 2068.866398870945 Miles Metros Cubicos
Sierra Chata Caudal Instantáneo Area Miles Metros Cubicos
1/2/2022 14:29: